# PyDESeq2 step-by-step pipeline

This notebook details all the steps of the PyDESeq2 pipeline.

It allows you to run the PyDESeq2 pipeline on the synthetic data provided in this repository.

In [ ]:
import os
import pickle as pkl

from pydeseq2.DeseqDataSet import DeseqDataSet
from pydeseq2.DeseqStats import DeseqStats
from pydeseq2.utils import load_example_data

In [ ]:
SAVE = False  # whether to save the outputs of this notebook

## Data loading

In [ ]:
OUTPUT_PATH = f"../output_files/synthetic_example"  # Replace this with the path were you wish to save outputs
os.makedirs(OUTPUT_PATH, exist_ok=True)  # Create path if it doesn't exist

In [ ]:
counts_df = load_example_data(
    modality="raw_counts",
    dataset="synthetic",
    debug=False,
)

In [ ]:
clinical_df = load_example_data(
    modality="clinical",
    dataset="synthetic",
    debug=False,
)

In [ ]:
counts_df

In [ ]:
clinical_df

Filter out genes that have less than 10 counts in total.
There shouldn't be any in the synthetic dataset, but pre-filtering genes is good practice in general.

In [ ]:
genes_to_keep = counts_df.columns[counts_df.sum(axis=0) >= 10]
len(genes_to_keep)

In [ ]:
counts_df = counts_df[genes_to_keep]

## 1 - Read counts modeling with the `DeseqDataSet` class

The `DeseqDataSet` class has two mandatory arguments, `counts_df` and `clinical_df`, as well as a set of optional keyword arguments, among which:

- `design_factor`: the name of the column of clinical to be used as a design variable
- `refit_cooks`: whether to refit cooks outliers – this is advised, in general.

Note: in the case of the provided synthetic data, there won't be any Cooks outliers.

In [ ]:
dds = DeseqDataSet(
    counts_df,
    clinical_df,
    design_factors="condition",  # compare samples based on the "condition" column ("B" vs "A")
    refit_cooks=True,
    n_cpus=8,
)

## Compute normalization factors

In [ ]:
dds.fit_size_factors()

In [ ]:
dds.size_factors

## Fit genewise dispersions

In [ ]:
dds.fit_genewise_dispersions()

In [ ]:
dds.genewise_dispersions

## Fit dispersion trend coefficients

In [ ]:
dds.fit_dispersion_trend()

In [ ]:
dds.trend_coeffs

In [ ]:
dds.fitted_dispersions

## Dispersion priors

In [ ]:
dds.fit_dispersion_prior()

In [ ]:
print(f"logres_prior={dds._squared_logres}, sigma_prior={dds.prior_disp_var}")

## MAP Dispersions

The `fit_MAP_dispersions` method filters the genes for which dispersion shrinkage is applied.  
Indeed, for genes whose MLE dispersions are too high above the trend curve, the original MLE value is kept.  
The final values of the dispersions that are used for downstream analysis is stored in `dds.dispersions`.

In [ ]:
dds.fit_MAP_dispersions()

In [ ]:
dds.MAP_dispersions

In [ ]:
dds.dispersions

## Fit log fold changes

Note that in the `DeseqDataSet` object, the log-fold changes are stored in natural log scale,
but that the results dataframe output by the `summary` method of `DeseqStats` displays LFCs in log2 scale (see later on).

In [ ]:
dds.fit_LFC()

In [ ]:
dds.LFCs

### Calculate Cooks distances and refit (optional) 

In [ ]:
dds.calculate_cooks()

In [ ]:
if dds.refit_cooks:
    # Replace outlier counts
    dds.refit()

In [ ]:
### Save everything

if SAVE:
    with open(os.path.join(OUTPUT_PATH, "dds_detailed_pipe.pkl"), "wb") as f:
        pkl.dump(dds, f)

## 2 - Statistical analysis with the `DeseqStats` class

The `DeseqDataSet` class has a unique mandatory arguments, `dds`, which should be a *fitted* `DeseqDataSet` object, as well as a set of optional keyword arguments, among which:

- `alpha`: the p-value and adjusted p-value significance threshold
- `cooks_filter`: whether to filter p-values based on cooks outliers
- `independent_filter`: whether to perform independent filtering to correct p-value trends.

In [ ]:
stat_res = DeseqStats(dds, alpha=0.05, cooks_filter=True, independent_filter=True)

## Wald tests

In [ ]:
stat_res.run_wald_test()

In [ ]:
stat_res.p_values

### Cooks filtering (optional)

Note: in the case of the provided synthetic data, there won't be any outliers.

In [ ]:
if stat_res.cooks_filter:
    stat_res._cooks_filtering()
stat_res.p_values

## P-value adjustment

In [ ]:
if stat_res.independent_filter:
    stat_res._independent_filtering()
else:
    stat_res._p_value_adjustment()

In [ ]:
stat_res.padj

## Building a results dataframe

This dataframe is stored in the `results_df` attribute of the `DeseqStats` class.

In [ ]:
stat_res.summary()

In [ ]:
### Save everything

if SAVE:
    with open(os.path.join(OUTPUT_PATH, "stat_results_detailed_pipe.pkl"), "wb") as f:
        pkl.dump(stat_res, f)

## LFC Shrinkage

For visualization or post-processing purposes, it might be suitable to perform LFC shrinkage. This is implemented by the `lfc_shrink` method.

In [ ]:
stat_res.lfc_shrink()

In [ ]:
### Save everything

if SAVE:
    with open(
        os.path.join(OUTPUT_PATH, "shrunk_stat_results_detailed_pipe.pkl"), "wb"
    ) as f:
        pkl.dump(stat_res, f)